Kod složenijih primjera `split()` metoda brzo postaje neučinkovita. Dohvatimo linkove pomoću regularnih izraza.

> ZADATAK 3. Dohvaćanje linkova korištenjem biblioteke [re](https://docs.python.org/3/library/re.html); primjer: 

`<a href="/images/nastava/izvedbeni/2023_2024/PDS/1/DINP_FIDIT_2023_2024_OPROG.pdf" target="_blank"><i class="fa fa-file-pdf-o"></i></a></td>`



Metode `re.search()`, `re.match()`, `re.fullmatch()`:
- [re.match()](https://docs.python.org/3/library/re.html#re.match) checks for a match only at the beginning of the string
- [re.search()](https://docs.python.org/3/library/re.html#re.search) checks for a match anywhere in the string (this is what Perl does by default)
- [re.fullmatch()](https://docs.python.org/3/library/re.html#re.fullmatch) checks for entire string to be a match

- [re.compile()](https://docs.python.org/3/library/re.html#re.compile) compiles regular expression

- [re.findall()](https://docs.python.org/3/library/re.html#re.findall) returns all non-overlapping matches of pattern in string, as a list of strings or tuples. 


> ZADATAK 3a Sa Wikipedia članka o metodologiji [MIRIS](https://hr.wikipedia.org/wiki/MIRIS_(ra%C4%8Dunarstvo)) saznajte:

>> Koliko puta se pojavljuje riječ MIRIS?

>> Koliko puta se spominje prezime koje završava na "ić"? Spremite rezultat u listu.

>> Izlučite sve linkove sa članka.

In [ ]:
import requests

url = "https://hr.wikipedia.org/wiki/MIRIS_(ra%C4%8Dunarstvo)"

page = requests.get(url)

print(page.text)

Web sjedište upozorava nas da poštujemo robots.txt te da definiramo zaglavlje naših zahtjeva - agenta koji šalje upit:

In [ ]:
import requests

url = "https://hr.wikipedia.org/wiki/MIRIS_(ra%C4%8Dunarstvo)"

headers = {
    'User-Agent': 'UTOR Agent',
    'From': 'utor@agent.hr'  
}

page = requests.get(url, headers=headers)

print(page.text)

In [ ]:
import re

# Pojavljivanje riječi MIRIS

result31 = re.findall(r"MIRIS", page.text)

print(f"Riječ MIRIS pojavljuje se: {len(result31)} puta")

In [ ]:
# Pojavljivanje ić prezimena

result32 = re.findall(r"\s*([A-Za-zČĆŠĐŽčćšđž]+ić)[ \.\,]", page.text)
print("U tekstu se prezime na \"ić\" pojavljuje: {} puta".format(len(result32)))

In [ ]:
# Izlučivanje linkova

regex33 = re.compile(r"(https:\/\/([\w\d-]+\.|[\w\d-]+\/)+)")
result33 = re.findall(regex33, page.text)

print([r[0] for r in result33])

Izlučivanje informacija relativno je kompleksan zadatak, posebice kada govorimo o ciljevima kao što su URL, adresa e-pošte ili imena osoba.

#### Preuzimanje izvedbenih planova Fakulteta za Fiziku 

Preuzmimo izvedbene programe [Fakulteta za Fiziku](https://www.phy.uniri.hr/hr/).

Svi izvedbeni programi nalaze se na stranici: https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html .

In [ ]:
import requests

URL_DINP = "https://phy.uniri.hr/nastava/izvedbeni-programi/"

page_dinp = requests.get(URL_DINP)

print(page_dinp.text)

Razlomimo tablicu na retke pomoću `.split("<tr>")` te pronađimo sve `href` vrijednosti.

In [ ]:
# Primjer nad svi retcima tablice
retci_tablice = page_dinp.text.split("<tr>")
regex = re.compile(r"href=\"([\w\/ -_\.]*)\"")


linkovi = []
for redak in retci_tablice:
    try:
        linkovi.append(re.search(regex, redak).group(1))
        print(re.search(regex, redak).group(1))
    except AttributeError:
        print("")

In [ ]:
linkovi

#### Dohvaćanje podataka pomoću biblioteke BeautifulSoup

> ZADATAK 4. Dohvaćanje podataka o nazivu kolegija i nositelju pomoću html oznaka

[BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) je Python biblioteka za dohvaćanje podataka iz HTML/XML datoteka. BeautifulSoup podržava mnogobrojne [parsere](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser):

| Parser | Typical usage | Advantages | Disadvantages |
| ------ | ------------- | ---------- | ------------- |
| Python’s html.parser | `BeautifulSoup(markup, "html.parser")`| - Batteries included <br> - Decent speed | - Not as fast as lxml, less lenient than html5lib. |
| [**lxml’s HTML parser**](https://lxml.de/) | `BeautifulSoup(markup, "lxml")` | - Very fast | - External C dependency |
| lxml’s XML parser | `BeautifulSoup(markup, "lxml-xml") BeautifulSoup(markup, "xml")` | - Very fast <br> - The only currently supported XML parser | - External C dependency |
| html5lib | `BeautifulSoup(markup, "html5lib")` | - Extremely lenient <br> - Parses pages the same way a web browser does <br> - Creates valid HTML5 | - Very slow <br> - External Python dependency |





##### Osnovno navigiranje pomoću BS4


Dohvatimo za početak stranicu s izvedbenim planovima diplomskog studija: `https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html`

In [ ]:
URL_DINP = "https://phy.uniri.hr/nastava/izvedbeni-programi/"

page_dinp = requests.get(URL_DINP)

print(page_dinp.text)

Stvorimo novi soup objekt pomoću BeautifulSoup konstruktora:

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_dinp.text, 'html.parser')

print(soup.prettify())

In [ ]:
# Sadržaj pod "title" HTML oznakom
print(soup.title)

# Ime HTML oznake
print(soup.title.name)

# Sadržaj HTML oznake
print(soup.title.string)

# Ime "nadređenog" html elementa
print(soup.title.parent.name)

# Prvi element u stablu s oznakom p - paragraph
print(soup.p)

# a element unutar prvog paragraph elementa
print(soup.p.a)

# Prvi a element unutar prvog paragrapha
print(soup.p.find_all("a", limit=1))
# ili
print(soup.p.find("a"))

# Href anchor elementa
print(soup.p.a["href"])

# Prvi element s a - anchor oznakom
print(soup.a)

# Svi elemeti sa span oznakom
print(soup.find_all('span'))

# Prvi element s danim id parametrima
soup.find(id="current")

> ZADATAK 5.  Pronađite element kojem je klasa "menu-item-3297".

In [ ]:
# class je ključna riječ koja je rezervirana za definiranje klase!
print(soup.find(class_="menu-item-3297"))

> ZADATAK 6. Pronađite sve elemente kojima je klasa "menu-item-N", pri čemu je N troznamenkasti broj.

In [ ]:
import re
print(soup.find_all(class_=re.compile(r"menu-item-[0-9]{3}(?!\d)")))

> ZADATAK 3* 

In [ ]:
linkovi = []
retci = soup.find_all('td')
for r in retci:
    try:
        print(r.find('a')["href"])
        linkovi.append(r.find('a')["href"])
    except:
        pass

#### Preuzimanje sadržaja (PDF)

Do sada smo dohvatili sve URL-ove koji su nam interesantni (`linkovi`), potrebno je dostupne (.pdf) datoteke pohraniti na lokalno računalo/disk -> Ne želimo isponova dohvaćati podatke kada za to postoji potreba!


> ZADATAK 7. Preuzmite PDF dokumente na lokalno računalo.

In [ ]:
# Spajanje listi
pdf_url = linkovi

# Čišćenje duplikata - gubi se poredak
pdf_url = list(set(pdf_url))

# Zanimaju nas isključivo URL-ovi PDF-a
pdf_url = [p for p in pdf_url if p.endswith(".pdf")]

print(len(pdf_url))
print(pdf_url[1:6])

Primjer:

In [ ]:
primjer_url = pdf_url[10]

with open(f'./DINPovi/{primjer_url.split("/")[-1]}', 'xb') as f:
    f.write(requests.get(primjer_url).content)

In [ ]:
from random import random
from time import sleep


for link in pdf_url:
    sleep(random()) # Nasumični delay od 0 do 1 sekunde
    url = link
    print(url)
    try:
        with open(f'./DINPovi/{url.split("/")[-1]}', 'xb') as f:
            f.write(requests.get(link).content)
    except FileExistsError:
        print("Already downloaded")